In [9]:
import os
import cv2
import numpy as np
import mahotas
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from skimage.feature import hog

# Load images and labels
base_path = 'C:/Users/Aakansh/Lung Classification/Lung-X-Ray-Project/Database/'
categories = ['Lung_Opacity', 'Normal', 'Viral Pneumonia']
data, labels = [], []

for category in categories:
    path = os.path.join(base_path, category)
    label = category
    for img_name in os.listdir(path):
        img_path = os.path.join(path, img_name)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Load in grayscale
        img = cv2.resize(img, (128, 128))  # Resize to a fixed size
        data.append(img)
        labels.append(label)

# Convert to arrays
data = np.array(data)
labels = np.array(labels)

# Encode labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [11]:
def preprocess_image(image):
    """ Apply sharpening and histogram equalization """
    hist_eq = cv2.equalizeHist(image)
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    sharpened = cv2.filter2D(hist_eq, -1, kernel)
    return sharpened

# Preprocess images
X_train_processed = np.array([preprocess_image(img) for img in X_train])
X_test_processed = np.array([preprocess_image(img) for img in X_test])

In [13]:
def extract_sift_features(images, max_features=128):
    sift = cv2.SIFT_create()
    features = []
    for img in images:
        keypoints, descriptors = sift.detectAndCompute(img, None)
        if descriptors is None:
            descriptors = np.zeros((1, max_features))
        flattened = descriptors.flatten()
        flattened = np.pad(flattened, (0, max(0, max_features - len(flattened))), mode='constant')[:max_features]
        features.append(flattened)
    return np.array(features)

def extract_hog_features(images):
    features = []
    for img in images:
        hog_features = hog(img, orientations=9, pixels_per_cell=(8, 8),
                           cells_per_block=(2, 2), block_norm='L2-Hys', visualize=False)
        features.append(hog_features)
    return np.array(features)

def extract_lbp_features(images):
    features = []
    for img in images:
        lbp = mahotas.features.lbp(img, radius=2, points=16, ignore_zeros=False)
        features.append(lbp)
    return np.array(features)

# Extract Features
sift_features = extract_sift_features(X_train_processed)
hog_features = extract_hog_features(X_train_processed)
lbp_features = extract_lbp_features(X_train_processed)
X_train_features = np.hstack((sift_features, hog_features, lbp_features))

# Repeat for test data
sift_test_features = extract_sift_features(X_test_processed)
hog_test_features = extract_hog_features(X_test_processed)
lbp_test_features = extract_lbp_features(X_test_processed)
X_test_features = np.hstack((sift_test_features, hog_test_features, lbp_test_features))

In [15]:
# Standardize features
scaler = StandardScaler()
X_train_features = scaler.fit_transform(X_train_features)
X_test_features = scaler.transform(X_test_features)

# Train logistic regression model
model = LogisticRegression(max_iter=1000, random_state=42, solver='lbfgs', C=10)
model.fit(X_train_features, y_train)

# Predict on test set
y_pred = model.predict(X_test_features)

# Evaluate model
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.82      0.84       248
           1       0.83      0.82      0.82       238
           2       0.92      0.99      0.95       209

    accuracy                           0.87       695
   macro avg       0.87      0.88      0.87       695
weighted avg       0.87      0.87      0.87       695

Accuracy: 0.8705035971223022


In [17]:
# Predict a new image
def preprocess_and_extract(image):
    image = preprocess_image(image)
    sift_features = extract_sift_features([image])
    hog_features = extract_hog_features([image])
    lbp_features = extract_lbp_features([image])
    features = np.hstack((sift_features, hog_features, lbp_features))
    return scaler.transform(features)

new_image = cv2.imread('../Sample_Test/LO/3.jpg', cv2.IMREAD_GRAYSCALE)
new_image = cv2.resize(new_image, (128, 128))
new_features = preprocess_and_extract(new_image)
prediction = model.predict(new_features)
print("Predicted Class:", label_encoder.inverse_transform(prediction))


Predicted Class: ['Lung_Opacity']
